# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
city_data = pd.read_csv("../WeatherPy/output/cities.csv")
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Klaksvík,62.23,-6.59,50.00,100,34,13.87,FO,1592804066
1,Ushuaia,-54.80,-68.30,33.80,86,28,6.93,AR,1592803623
2,Nanortalik,60.14,-45.24,45.18,75,74,5.82,GL,1592803629
3,Rikitea,-23.12,-134.97,70.34,68,71,17.09,PF,1592803619
4,Ballina,-28.87,153.57,66.20,39,0,13.87,AU,1592804066


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
gmaps.configure(api_key=g_key)
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

In [31]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), 
             point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
narrowed_df = city_data.loc[(city_data["Wind Speed"] <= 10) & (city_data["Cloudiness"] == 0) & \
                                   (city_data["Max Temp"] >= 70) & (city_data["Max Temp"] <= 80)].dropna()

narrowed_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Waddān,29.16,16.14,72.66,45,0,8.16,LY,1592804067
114,Arraial do Cabo,-22.97,-42.03,72.52,83,0,7.67,BR,1592803621
121,Porto Nacional,-10.71,-48.42,71.60,53,0,6.93,BR,1592804079
198,Adana Province,36.98,35.29,78.80,78,0,4.70,TR,1592803920
293,Pout,14.77,-17.06,78.80,83,0,9.17,SN,1592804098
329,São João da Barra,-21.64,-41.05,70.29,88,0,7.43,BR,1592803624
330,Tigzirt,36.89,4.12,73.90,52,0,7.20,DZ,1592804102
383,Pentecoste,-3.79,-39.27,75.00,82,0,1.99,BR,1592804108
416,Cabra,37.47,-4.44,71.01,42,0,3.15,ES,1592804112


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = narrowed_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Waddān,29.16,16.14,72.66,45,0,8.16,LY,1592804067,
114,Arraial do Cabo,-22.97,-42.03,72.52,83,0,7.67,BR,1592803621,
121,Porto Nacional,-10.71,-48.42,71.60,53,0,6.93,BR,1592804079,
198,Adana Province,36.98,35.29,78.80,78,0,4.70,TR,1592803920,
293,Pout,14.77,-17.06,78.80,83,0,9.17,SN,1592804098,
329,São João da Barra,-21.64,-41.05,70.29,88,0,7.43,BR,1592803624,
330,Tigzirt,36.89,4.12,73.90,52,0,7.20,DZ,1592804102,
383,Pentecoste,-3.79,-39.27,75.00,82,0,1.99,BR,1592804108,
416,Cabra,37.47,-4.44,71.01,42,0,3.15,ES,1592804112,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

hotel_df

Missing field/result... skipping.
------------
Closest hotel in Arraial do Cabo is Hotel da Canoa.
------------
Closest hotel in Porto Nacional is Vila Real Hotel.
------------
Closest hotel in Adana Province is Sheraton Grand Adana.
------------
Missing field/result... skipping.
------------
Closest hotel in São João da Barra is Pousada Kactus.
------------
Closest hotel in Tigzirt is Hôtel Mizrana.
------------
Closest hotel in Pentecoste is Pentecoste Palace Hotel.
------------
Closest hotel in Cabra is Hotel Villa María.
------------


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Waddān,29.16,16.14,72.66,45,0,8.16,LY,1592804067,
114,Arraial do Cabo,-22.97,-42.03,72.52,83,0,7.67,BR,1592803621,Hotel da Canoa
121,Porto Nacional,-10.71,-48.42,71.60,53,0,6.93,BR,1592804079,Vila Real Hotel
198,Adana Province,36.98,35.29,78.80,78,0,4.70,TR,1592803920,Sheraton Grand Adana
293,Pout,14.77,-17.06,78.80,83,0,9.17,SN,1592804098,
329,São João da Barra,-21.64,-41.05,70.29,88,0,7.43,BR,1592803624,Pousada Kactus
330,Tigzirt,36.89,4.12,73.90,52,0,7.20,DZ,1592804102,Hôtel Mizrana
383,Pentecoste,-3.79,-39.27,75.00,82,0,1.99,BR,1592804108,Pentecoste Palace Hotel
416,Cabra,37.47,-4.44,71.01,42,0,3.15,ES,1592804112,Hotel Villa María


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))